# BCS Data Analysis Project

To frame our analysis, we must start with a concrete question to ground ourselves.
In this case, we're asking ourselves; So what's up with counter heals?
* how many do decks play on average?
* how do high performing decks relate to this average?
* how about low performing decks?

In order to answer this we'll need to:
* obtain HTML page for each decklog
* add each deck's info. into a data base (each card name and it's quantity)
* determine the names we want to know about
* query and graph the data

In [ ]:
### Pipeline 1

## Obtain HTML

In [126]:

import pandas as pd

import requests
from bs4 import BeautifulSoup

URL = "https://en.cf-vanguard.com/event/bcs2526/bcs2526-california/"

soup = BeautifulSoup(requests.get(URL).text)

table investigation

In [127]:
# print(soup.find_all("table"))
## standard results are the first table, so that's what we'll focus on
print(soup.table.find_all("th"))

[<th style="text-align:center">Rank</th>, <th>Player</th>, <th>Deck</th>, <th style="text-align:center">Wins</th>, <th>Nation</th>]


In [128]:
def extract_header(header):
    return header.contents[0]

column_headers = []
ths = soup.table.find_all("th")
for th in ths:
    column_headers.append(extract_header(th))


In [129]:
print(column_headers)
print (rows[0])
print("----------")
print (rows[1])
print("----------")
print((rows[4]))

['Rank', 'Player', 'Deck', 'Wins', 'Nation']
<tr class="decklog-frame-trigger" data-aos="fade-up" data-deck-id="146MF">
<td class="rank-cell rank-1">1st</td>
<td colspan="2" style="border-right: 1.5px solid #2c3e50">
<div class="deck-preview"><img alt="Click to view deck" src="/wordpress/wp-content/uploads/haomai-flair-banner.png"/></div>
</td>
<td class="win-ratio">11</td>
<td style="padding: 10px">
<div class="nation-info"><img alt="Brandt Gate" class="nation-flag" src="/wordpress/wp-content/uploads/Brandt-Gate-flag-deck.png"/><span class="nation-name">Brandt Gate</span></div>
</td>
</tr>
----------
<tr class="decklog-frame-trigger" data-aos="fade-up" data-deck-id="6EZB7">
<td class="rank-cell rank-2">2nd</td>
<td colspan="2" style="border-right: 1.5px solid #2c3e50">
<div class="deck-preview"><img alt="Click to view deck" src="/wordpress/wp-content/uploads/Minkang-flair-banner.png"/></div>
</td>
<td class="win-ratio">11</td>
<td style="padding: 10px">
<div class="nation-info"><img a

In [130]:
##### after some analysis, the grade 3 for top 3 decks can't be extracted rn, so we'll do it at the next stage, when we would add "NextStage"

In [135]:
def get_values(ddict, row):
    i = 0
    values=[]
    for item in row:
        if i == 1:
            if item.div:
                values.append( "Champion")
                i+=1
                continue
            # else:
            #     values.append(row[i].text)
            #     i+=1
            #     continue

        values.append(row[i].text)
        i+=1
    # Magic number hack :/
    if len(values) == 5: values.pop(2)
    return values

rows = soup.table.find_all('tr')
# Remove header row, so all rows have td, and not th
rows.pop(0)

dataDict = dict()
for row in rows:
    # Decklog as the key
    key = row['data-deck-id']
    values = get_values(dataDict, row.find_all('td'))
    values.append(key)

    dataDict[key] = values
  

_________________
['1st', 'Champion', '11', '\nBrandt Gate\n', '146MF']
_________________
['2nd', 'Champion', '11', '\nDark States\n', '6EZB7']
_________________
['3rd', 'Champion', '10', '\nStoicheia\n', '42N7X']
_________________
['4th', 'Ken', '10', '\nStoicheia\n', '1AQ5W']
_________________
['5th', 'Kay', '8', '\nDragon Empire\n', '6B3RK']
_________________
['6th', 'Arcagian', '8', '\nDark States\n', '430LM']


In [162]:
df = pd.DataFrame(dataDict).transpose()
df.head()

# df = df.set_axis([
#             'rank',
#             'name',
#             "wins",
#             'nation',
#             'decklog',
#                  ], axis='index)
# df.drop([0])
# df.head()

,0,1,2,3,4
decklog,0,0,0,0,0
146MF,1st,Champion,11,\nBrandt Gate\n,146MF
6EZB7,2nd,Champion,11,\nDark States\n,6EZB7
42N7X,3rd,Champion,10,\nStoicheia\n,42N7X
1AQ5W,4th,Ken,10,\nStoicheia\n,1AQ5W


In [145]:
### Pipeline 2 

In [152]:
URL2 = "https://decklog-en.bushiroad.com/view/146MF"

soup2 = BeautifulSoup(requests.get(URL2).text)
soup2.table

In [153]:
soup2.h